In [559]:
import numpy as np
from basis_states import *
from hamiltonian import *
import symmer as sym

In [742]:
N = 2
a, g, m = 1, 1, 1

In [743]:
H(N)

 2.188+0.000j IIII +
 0.500+0.000j IIIZ +
 0.500+0.000j IIZI +
-0.500+0.000j IZII +
-0.500+0.000j ZIII +
-0.188+0.000j ZZII +
-0.250+0.000j IXZX +
-0.250+0.000j IYZY +
-0.250+0.000j XZXI +
-0.250+0.000j YZYI

In [744]:
mesons, baryons = get_basis(N)

In [745]:
mesons, baryons

([ 1.000+0.000j |0011>,
   1.000+0.000j |0110>,
   1.000+0.000j |1001>,
   1.000+0.000j |1100>],
 [ 1.000+0.000j |0000>])

In [746]:
Qx = Qx_tot(N)
Qy = Qy_tot(N)

Qz = Qz_tot(N)

In [747]:
Qx, Qy

( 0.250+0.000j IIXX +
  0.250+0.000j IIYY +
  0.250+0.000j XXII +
  0.250+0.000j YYII,
 -0.250+0.000j IIXY +
  0.250+0.000j IIYX +
 -0.250+0.000j XYII +
  0.250+0.000j YXII)

In [748]:
Qx.dagger == Qx

True

In [749]:
Qx.commutes(Qy)

False

In [750]:
qxmat = Qx.to_sparse_matrix.toarray()

In [751]:
vals, vecs = np.linalg.eigh(qxmat)

In [752]:
epsilon = 1e-5
vals[np.abs(vals) < epsilon] =0
vals

array([-1. , -0.5, -0.5, -0.5, -0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0.5,  0.5,  0.5,  0.5,  1. ])

In [753]:
np.where(vals == 0)[0]

array([ 5,  6,  7,  8,  9, 10])

In [754]:
for index in np.where(vals == 0)[0]:
    vecs[:, index][np.abs(vecs[:, index]) < epsilon] = 0
    psi = sym.QuantumState.from_array(vecs[:, index].reshape([-1, 1]))
    if list((Qz_tot(N) * psi).to_dictionary.values())[0] == 0:
        print(psi, "\n")

 1.000+0.000j |0000> 

 1.000+0.000j |0011> 

 1.000+0.000j |1100> 

 1.000+0.000j |1111> 

 0.707+0.000j |0110> +
-0.707+0.000j |1001> 



In [755]:
index = 9
vecs[:, index][np.abs(vecs[:, index]) < epsilon] = 0
vecs[:, index]

array([ 0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.70710678+0.j,  0.        +0.j,
        0.        +0.j, -0.70710678+0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j])

In [756]:
psi = sym.QuantumState.from_array(vecs[:, index].reshape([-1, 1]))
psi

 0.707+0.000j |0110> +
-0.707+0.000j |1001>

In [757]:
Qy_tot(N) * psi 


In [758]:
Qx * psi

In [759]:
Qz = Qz_tot(N)
Qz

-0.250+0.000j IIIZ +
 0.250+0.000j IIZI +
-0.250+0.000j IZII +
 0.250+0.000j ZIII

In [760]:
mesons

[ 1.000+0.000j |0011>,
  1.000+0.000j |0110>,
  1.000+0.000j |1001>,
  1.000+0.000j |1100>]

In [761]:
B(N)

 0.250+0.000j IIIZ +
 0.250+0.000j IIZI +
 0.250+0.000j IZII +
 0.250+0.000j ZIII

In [762]:
(-1)**bitstrings * 1/4

array([[ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25, -0.25],
       [ 0.25,  0.25, -0.25,  0.25],
       [ 0.25,  0.25, -0.25, -0.25],
       [ 0.25, -0.25,  0.25,  0.25],
       [ 0.25, -0.25,  0.25, -0.25],
       [ 0.25, -0.25, -0.25,  0.25],
       [ 0.25, -0.25, -0.25, -0.25],
       [-0.25,  0.25,  0.25,  0.25],
       [-0.25,  0.25,  0.25, -0.25],
       [-0.25,  0.25, -0.25,  0.25],
       [-0.25,  0.25, -0.25, -0.25],
       [-0.25, -0.25,  0.25,  0.25],
       [-0.25, -0.25,  0.25, -0.25],
       [-0.25, -0.25, -0.25,  0.25],
       [-0.25, -0.25, -0.25, -0.25]])

In [763]:
## speedup choose Qz bitrings  by rules determined by combinations
# (number of spin up == spin down!)
n_qubits = 2*N

bitstrings = (
                (
                        np.arange(2 ** n_qubits).reshape([-1, 1]) &
                        (1 << np.arange(n_qubits))[::-1]
                ) > 0).astype(int)

sign_array = np.array([(-1)**(i%2) for i in range(n_qubits)])
Qz_mask = np.sum(bitstrings*sign_array, axis=1) == 0

## speedup by slicing bitstrings with Qz_mask!!!

B_arr = (-1)**bitstrings * 1/4
B_mask = np.logical_or(np.sum(B_arr, axis=1) == 0, np.sum(B_arr, axis=1) == 1)

print('valid B states:', np.sum(B_mask))
print('valid Qz states:', np.sum(Qz_mask))
QzandB_mask = np.logical_and(B_mask, Qz_mask)
print('valid Qz AND B states:', np.sum(QzandB_mask))
print('out of a possible:', 2**(2*N))
valid_bitstrings = bitstrings[QzandB_mask, :]
valid_bitstrings

valid B states: 7
valid Qz states: 6
valid Qz AND B states: 5
out of a possible: 16


array([[0, 0, 0, 0],
       [0, 0, 1, 1],
       [0, 1, 1, 0],
       [1, 0, 0, 1],
       [1, 1, 0, 0]])

In [764]:
bitstrings[B_mask, :][0]

array([0, 0, 0, 0])

In [765]:
np.sum((bitstrings*1/4)[-2:,:], axis=1)

array([0.75, 1.  ])

In [766]:
for state in valid_bitstrings:
    psi = sym.QuantumState(state, [1])
    print(psi.dagger * B(N) * psi)

(1+0j)
0j
0j
0j
0j


In [767]:
Qy

-0.250+0.000j IIXY +
 0.250+0.000j IIYX +
-0.250+0.000j XYII +
 0.250+0.000j YXII

In [768]:
help(QT.symmetry_generators.rotate_onto_single_qubit_paulis)

Help on method rotate_onto_single_qubit_paulis in module symmer.operators.independent_op:

rotate_onto_single_qubit_paulis() -> 'IndependentOp' method of symmer.operators.independent_op.IndependentOp instance
    Returns the rotated single-qubit Pauli stabilizers.
    
    Returns: 
        Rotated single-qubit Pauli stabilizers.



In [769]:
from symmer.projection import QubitTapering

QT_X = QubitTapering(Qx, target_sqp='Z')
# print(QT.symmetry_generators)
print(QT_X.symmetry_generators.rotate_onto_single_qubit_paulis())

stabilizer_rotations = QT_X.symmetry_generators.stabilizer_rotations

Qx_inZ_basis = Qx.perform_rotations(stabilizer_rotations)
Qx_inZ_basis

-1 IIIZ 
1 IIZI 
-1 IZII 
1 ZIII


 0.250-0.000j IIZI +
 0.250-0.000j IIZZ +
 0.250-0.000j ZIII +
 0.250-0.000j ZZII

In [770]:
Qy


-0.250+0.000j IIXY +
 0.250+0.000j IIYX +
-0.250+0.000j XYII +
 0.250+0.000j YXII

In [771]:
from symmer.projection import QubitTapering

QT_Y = QubitTapering(Qy, target_sqp='Z')
# print(QT.symmetry_generators)
print(QT_Y.symmetry_generators.rotate_onto_single_qubit_paulis())

stabilizer_rotations_Y = QT_Y.symmetry_generators.stabilizer_rotations

Qy_inZ_basis = Qy.perform_rotations(stabilizer_rotations_Y)
Qy_inZ_basis

-1 IIIZ 
-1 IIZI 
-1 IZII 
-1 ZIII


 0.250-0.000j IIIZ +
 0.250-0.000j IIZZ +
 0.250-0.000j IZII +
 0.250-0.000j ZZII

In [772]:
from symmer.evolution.exponentiation import exponentiate_single_Pop

In [774]:
R = sym.PauliwordOp.from_list(['I'*Qy.n_terms])
for index, (P_k, angle) in enumerate(stabilizer_rotations_Y[::-1]):
    angle = 1j*np.pi/4
    R*=exponentiate_single_Pop(P_k.multiply_by_constant(angle))

R * Qy * R.dagger

 0.250+0.000j IIIZ +
 0.250+0.000j IIZZ +
 0.250+0.000j IZII +
 0.250+0.000j ZZII

In [513]:
state = sym.QuantumState([[1, 0, 1, 0, 1, 1, 1, 1]])
Qy_inZ_basis * state


AssertionError: PauliwordOps defined for different number of qubits

In [ ]:
state_normal = R.dagger * state
Qy*state_normal

AssertionError: PauliwordOps defined for different number of qubits

In [ ]:
state_normal

 0.000+0.500j |0000> +
 0.500+0.000j |0011> +
-0.500+0.000j |1100> +
 0.000+0.500j |1111>

In [ ]:
odd_Qy_mask = np.array(list(zip([False]*N, [True]*N))).reshape([-1])
odd_Qy_mask

array([False,  True, False,  True, False,  True])

In [ ]:
bitstrings[]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]])

In [ ]:
Qz = Qz_tot(N)
Qz

-0.250+0.000j IIIIIIIIIIIIIIIZ +
 0.250+0.000j IIIIIIIIIIIIIIZI +
-0.250+0.000j IIIIIIIIIIIIIZII +
 0.250+0.000j IIIIIIIIIIIIZIII +
-0.250+0.000j IIIIIIIIIIIZIIII +
 0.250+0.000j IIIIIIIIIIZIIIII +
-0.250+0.000j IIIIIIIIIZIIIIII +
 0.250+0.000j IIIIIIIIZIIIIIII +
-0.250+0.000j IIIIIIIZIIIIIIII +
 0.250+0.000j IIIIIIZIIIIIIIII +
-0.250+0.000j IIIIIZIIIIIIIIII +
 0.250+0.000j IIIIZIIIIIIIIIII +
-0.250+0.000j IIIZIIIIIIIIIIII +
 0.250+0.000j IIZIIIIIIIIIIIII +
-0.250+0.000j IZIIIIIIIIIIIIII +
 0.250+0.000j ZIIIIIIIIIIIIIII

In [776]:
import itertools
allowed_Y = [(1,0), (1,1)]
product = list(itertools.product(allowed_Y, repeat=N))

arr = np.array(product)
arr = arr.reshape(arr.shape[0],-1)

arr

array([[1, 0, 1, 0],
       [1, 0, 1, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 1]])

In [783]:
arr

array([[1, 0, 1, 0],
       [1, 0, 1, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 1]])

In [786]:
solutions = {}
for i in range(arr.shape[0]):
    state =  sym.QuantumState(arr[i,:].reshape([1,-1]), [1]) 
    solutions[i] = R.dagger * state
    # solutions[i] = state

solutions

{0:  0.500+0.000j |0000> +
  0.000-0.500j |0011> +
  0.000-0.500j |1100> +
 -0.500+0.000j |1111>,
 1:  0.000+0.500j |0000> +
 -0.500+0.000j |0011> +
  0.500+0.000j |1100> +
  0.000+0.500j |1111>,
 2:  0.000+0.500j |0000> +
  0.500+0.000j |0011> +
 -0.500+0.000j |1100> +
  0.000+0.500j |1111>,
 3: -0.500+0.000j |0000> +
  0.000-0.500j |0011> +
  0.000-0.500j |1100> +
  0.500+0.000j |1111>}

In [ ]:
solutions[1] - solutions[3]

 0.500+0.500j |0000> +
-0.500+0.500j |0011> +
 0.500+0.500j |1100> +
-0.500+0.500j |1111>

In [ ]:
solutions[0]

 0.500+0.000j |0000> +
 0.000-0.500j |0011> +
 0.000-0.500j |1100> +
-0.500+0.000j |1111>

In [ ]:
solutions[0] + solutions[1] - solutions[2] + solutions[3]

-1.000-1.000j |0011> +
 1.000-1.000j |1100>

In [ ]:
ind = 1

state =  sym.QuantumState(arr[ind].reshape([1,-1]), [1]) 
R.dagger * statea

 0.000+0.500j |0000> +
-0.500+0.000j |0011> +
 0.500+0.000j |1100> +
 0.000+0.500j |1111>

In [ ]:
state =  sym.QuantumState(arr[3].reshape([1,-1]), [1]) 
Qy_inZ_basis * state

In [ ]:
Qy * R.dagger * state

In [ ]:
R.dagger * state

-0.500+0.000j |0000> +
 0.000-0.500j |0011> +
 0.000-0.500j |1100> +
 0.500+0.000j |1111>

In [ ]:

 
letters = ['10', '11']
 
# size of combination is set to 3
a = itertools.combinations(letters, 3) 
y = [' '.join(i) for i in a]
 
print(y)

[]


In [ ]:
lst = [2, 3]
combs = []

els = [list(x) for x in itertools.combinations(lst, N)]
els

[[2, 3]]

In [ ]:
lst

[2, 3, 2, 3]

In [ ]:
lst

[2, 3, 2, 3]

[('10', '10'), ('10', '11'), ('11', '10'), ('11', '11')]

[('10', '10'), ('10', '11'), ('11', '10'), ('11', '11')]

In [ ]:
Qy * sym.QuantumState([[1, 0, 0, 1]]) - Qy * sym.QuantumState([[0, 1, 1, 0]])

In [514]:
state1 = sym.QuantumState([[0, 1, 1, 0]])
state2 = sym.QuantumState([[1, 0, 0, 1]])

In [521]:
 Qx * (state1 - state2)

In [519]:
state1.dagger * Qy * state1

0j

In [553]:
Qy

-0.250+0.000j IIIIIIXY +
 0.250+0.000j IIIIIIYX +
-0.250+0.000j IIIIXYII +
 0.250+0.000j IIIIYXII +
-0.250+0.000j IIXYIIII +
 0.250+0.000j IIYXIIII +
-0.250+0.000j XYIIIIII +
 0.250+0.000j YXIIIIII

In [583]:
op  = Qy * Qx * Qz

In [584]:
# op

In [585]:
vals, vecs = np.linalg.eigh(op.to_sparse_matrix.toarray())
epsilon = 1e-12
mask = np.abs(vals)<epsilon


In [590]:
indices = np.where(mask)[0]
indices

array([ 45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187,
       188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,
       201, 202, 203, 204, 205, 206, 207, 208, 209, 210])

In [591]:
index = indices[0]

In [596]:
psi = sym.QuantumState.from_array(vecs[:, index].reshape([-1, 1]))
Qz * psi 

-0.001+0.002j |01001001> +
 0.000+0.002j |01001100> +
-0.033+0.016j |01011000> +
 0.006-0.005j |01011011> +
-0.018-0.014j |01011110> +
 0.003+0.002j |01100001> +
 0.025-0.007j |01100010> +
 0.058-0.007j |01100100> +
-0.011-0.076j |01100111> +
 0.226+0.060j |01101000> +
-0.005+0.022j |01101011> +
 0.019+0.032j |01101101> +
 0.038-0.001j |01101110> +
-0.250-0.047j |01110000> +
 0.019-0.028j |01110011> +
-0.023-0.001j |01110110> +
 0.049-0.013j |01111001> +
-0.029-0.096j |01111010> +
-0.064+0.018j |01111100> +
 0.018-0.042j |01111111> +
-0.045+0.018j |10000000> +
 0.007-0.005j |10000011> +
 0.001-0.002j |10000101> +
-0.000-0.002j |10000110> +
 0.000+0.002j |10001001> +
-0.003-0.002j |10010001> +
-0.009+0.004j |10010010> +
-0.026-0.009j |10010100> +
 0.005+0.081j |10010111> +
-0.112-0.036j |10011000> +
 0.002-0.011j |10011011> +
-0.001-0.019j |10011101> +
 0.014-0.039j |10011110> +
-0.016+0.002j |10100001> +
-0.114-0.024j |10100100> +
 0.003-0.011j |10100111> +
-0.052+0.040j |10101101> +
-

In [597]:
import scipy as sp

In [611]:
Qy_mat = Qy.to_sparse_matrix.toarray()
Qy_ns = sp.linalg.null_space(Qy_mat)

# np.allclose(Qy_mat@Qy_ns[:,3], np.zeros(Qy_ns.shape[0]))

Qz_mat = Qz.to_sparse_matrix.toarray()
Qz_ns = sp.linalg.null_space(Qz_mat)

Qx_mat = Qx.to_sparse_matrix.toarray()
Qx_ns = sp.linalg.null_space(Qx_mat)

In [616]:
Qy_ns[:,3].shape, 2**(2*N)

((256,), 256)

In [644]:
op  = Qy + Qx + Qz
op_mat = op.to_sparse_matrix.toarray()
op_ns = sp.linalg.null_space(op_mat)

null_vecs = [sym.QuantumState.from_array(op_ns[:, i].reshape([-1,1])) for i in range(op_ns.shape[1])]

In [646]:
for vec in null_vecs:
    out_y = Qy*vec
    out_x = Qx*vec
    out_z = Qz*vec
    if out_x.n_terms== 0 and out_y.n_terms==0 and out_z.n_terms == 0:
        print(vec)


 1.000-0.000j |1100>
 1.000-0.000j |1111>
 1.000-0.000j |0011>
 1.000-0.000j |0000>


In [ ]:
op  = Qy * Qx * Qz

In [651]:
psi = sym.QuantumState([[1,0,0,1],
                  [0,1,1,0]], [1,-1]).normalize
psi

 0.707+0.000j |1001> +
-0.707+0.000j |0110>

In [653]:
Qy * psi

In [654]:
Qx * psi

In [655]:
Qz * psi

In [656]:
op*psi

In [692]:
op  = Qy + Qx +Qz

vals, vecs = np.linalg.eigh(op.to_sparse_matrix.toarray())
inds = np.where(np.abs(vals)<1e-12)[0]

In [693]:
valid_states = vecs[:, inds]

In [694]:
psi = sym.QuantumState.from_array(valid_states[:,0].reshape([-1,1]))

In [696]:
Qy*psi

 0.000+0.500j |0010>